In [1]:
import os
os.chdir("..")
os.getcwd()

'/root/code/Protenix'

In [2]:
INPUT_DIR = "/vepfs/fs_projects/yangsw/af3-dev/release_data/mmcif_bioassembly"
OUTPUT_LMDB_PATH = "/vepfs/fs_projects/yangsw/af3-dev/release_data/lmdb/mmcif_bioassembly.lmdb"
N_COMMIT = 1000
ENCODING = "utf-8"

In [3]:
import lmdb

env_new = lmdb.open(
    OUTPUT_LMDB_PATH,
    subdir=False,
    readonly=False,
    lock=False,
    readahead=False,
    meminit=False,
    max_readers=126,
    map_size=50 * 1024 * 1024 * 1024,
)

In [4]:
import gzip
import tqdm
from typing import Tuple, List

ba_file_names = os.listdir(INPUT_DIR)
todo_cif_names, finished_cif_names = [], []
with env_new.begin() as txn:
    with txn.cursor() as cursor:
        for file_name in tqdm.tqdm(ba_file_names, total=len(ba_file_names), ncols=60):
            cif_name = file_name[:4]
            if cursor.set_key(bytes(cif_name, ENCODING)):
                finished_cif_names.append(cif_name)
            else:
                todo_cif_names.append(cif_name)
print(f"\tTODO {len(todo_cif_names)}; FINISHED {len(finished_cif_names)}")

100%|███████████| 219912/219912 [00:01<00:00, 141837.68it/s]

	TODO 0; FINISHED 219912


In [5]:
txn_write = env_new.begin(write=True)
i = 0
for cif_name in tqdm.tqdm(todo_cif_names, total=len(todo_cif_names), ncols=60):
    key_bytes = bytes(cif_name, ENCODING)
    with open(os.path.join(INPUT_DIR, f"{cif_name}.pkl.gz"), "rb") as fp:
        value_bytes = fp.read()
    txn_write.put(key_bytes, value_bytes)
    i += 1
    if i % N_COMMIT == 0:
        txn_write.commit()
        txn_write = env_new.begin(write=True)

txn_write.commit()
env_new.close()

0it [00:00, ?it/s]


In [6]:
import pickle

env_new = lmdb.open(
    OUTPUT_LMDB_PATH,
    subdir=False,
    readonly=True,
    lock=False,
    readahead=False,
    meminit=False,
    max_readers=126,
    map_size=100 * 1024 * 1024 * 1024,
)

key = None
with env_new.begin() as txn:
    cursor = txn.cursor()
    for key, value in cursor:
        key = key.decode("utf-8")
        value = pickle.loads(gzip.decompress(value))
        print(key)
        print(value.keys())
        print(value["atom_array"][:20])
        break

100d
dict_keys(['pdb_id', 'sequences', 'release_date', 'assembly_id', 'num_assembly_polymer_chains', 'num_prot_chains', 'entity_poly_type', 'resolution', 'atom_array', 'num_tokens', 'msa_features', 'template_features', 'token_array'])
    A0      1  C   OP3    O         0.000    0.000    0.000
    A0      1  C   P      P         0.000    0.000    0.000
    A0      1  C   OP1    O         0.000    0.000    0.000
    A0      1  C   OP2    O         0.000    0.000    0.000
    A0      1  C   O5'    O        -4.549    5.095    4.262
    A0      1  C   C5'    C        -4.176    6.323    3.646
    A0      1  C   C4'    C        -3.853    7.410    4.672
    A0      1  C   O4'    O        -4.992    7.650    5.512
    A0      1  C   C3'    C        -2.713    7.010    5.605
    A0      1  C   O3'    O        -1.379    7.127    5.060
    A0      1  C   C2'    C        -2.950    7.949    6.756
    A0      1  C   O2'    O        -2.407    9.267    6.554
    A0      1  C   C1'    C        -4.489    

In [7]:
assert key is not None
with open(os.path.join(INPUT_DIR, f"{key}.pkl.gz"), "rb") as fp:
    value = pickle.loads(gzip.decompress(fp.read()))
    print(value.keys())
    print(value["atom_array"][:20])

dict_keys(['pdb_id', 'sequences', 'release_date', 'assembly_id', 'num_assembly_polymer_chains', 'num_prot_chains', 'entity_poly_type', 'resolution', 'atom_array', 'num_tokens', 'msa_features', 'template_features', 'token_array'])
    A0      1  C   OP3    O         0.000    0.000    0.000
    A0      1  C   P      P         0.000    0.000    0.000
    A0      1  C   OP1    O         0.000    0.000    0.000
    A0      1  C   OP2    O         0.000    0.000    0.000
    A0      1  C   O5'    O        -4.549    5.095    4.262
    A0      1  C   C5'    C        -4.176    6.323    3.646
    A0      1  C   C4'    C        -3.853    7.410    4.672
    A0      1  C   O4'    O        -4.992    7.650    5.512
    A0      1  C   C3'    C        -2.713    7.010    5.605
    A0      1  C   O3'    O        -1.379    7.127    5.060
    A0      1  C   C2'    C        -2.950    7.949    6.756
    A0      1  C   O2'    O        -2.407    9.267    6.554
    A0      1  C   C1'    C        -4.489    7.917